In [ ]:
# audio 
# encoder
# downsampling
# similar
# decoder
# loss

In [ ]:
from transformers import RobertaTokenizer, BertTokenizer, GPT2Tokenizer, LlamaTokenizerFast

In [2]:
import dac
from audiotools import AudioSignal

# Download a model
model_path = dac.utils.download(model_type="16khz")
model = dac.DAC.load(model_path)

model.to('cuda')

/raid/home/rajivratn/anaconda3/envs/last/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:30: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


DAC(
  (encoder): Encoder(
    (block): Sequential(
      (0): Conv1d(1, 64, kernel_size=(7,), stride=(1,), padding=(3,))
      (1): EncoderBlock(
        (block): Sequential(
          (0): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(3,))
              (2): Snake1d()
              (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
            )
          )
          (1): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(9,), dilation=(3,))
              (2): Snake1d()
              (3): Conv1d(64, 64, kernel_size=(1,), stride=(1,))
            )
          )
          (2): ResidualUnit(
            (block): Sequential(
              (0): Snake1d()
              (1): Conv1d(64, 64, kernel_size=(7,), stride=(1,), padding=(27,), dilation=(9,))
              (2): Snake1d()
              

In [21]:
# Load audio signal file
signal = AudioSignal('../data/input.wav')

# Encode audio signal as one long file
# (may run out of GPU memory on long files)
signal.to(model.device)

x = model.preprocess(signal.audio_data, signal.sample_rate)
z = model.encoder(x)

In [23]:
x.shape, x.shape[-1]/320, z.shape

(torch.Size([1, 1, 94720]), 296.0, torch.Size([1, 1024, 296]))

In [ ]:
# kernel size 9, stride 1, dilation 1, padding 4
# kernel size 9, stride 1, dilation 2, padding 8
# kernel size 9, stride 1, dilation 3, padding 12

In [ ]:

# Decode audio signal
y = model.decode(z)

# Alternatively, use the `compress` and `decompress` functions
# to compress long files.

signal = signal.cpu()
x = model.compress(signal)

# Save and load to and from disk
x.save("compressed.dac")
x = dac.DACFile.load("compressed.dac")

# Decompress it back to an AudioSignal
y = model.decompress(x)

# Write to file
y.write('output.wav')